# Airbus Uçak Sınıflandırma Projesi

Bu notebook, uçak görüntülerinin sınıflandırılması için veri önişleme, veri artırma, model oluşturma, değerlendirme, Grad-CAM ve hiperparametre optimizasyonu adımlarını içermektedir. Her adımda açıklamalar ve kodlar Türkçe olarak sunulmuştur.

---

Bu notebookta aşağıdaki ana başlıklar ve adımlar yer almaktadır:

1. **Veri Önişleme ve Görselleştirme**: Görüntülerin hazırlanması, etiketlenmesi, eğitim/doğrulama/test ayrımı ve veri analizi.
2. **Data Augmentation (Veri Çoğaltma)**: Görüntülerin çeşitli dönüşümlerle artırılması.
3. **CNN Modeli Oluşturma ve Eğitme**: Derin öğrenme tabanlı modelin kurulması ve eğitilmesi.
4. **Model Değerlendirme**: Başarım metrikleri, confusion matrix ve raporlar.
5. **Grad-CAM ile Görselleştirme**: Modelin odaklandığı bölgelerin görselleştirilmesi.
6. **Hiperparametre Optimizasyonu**: Model parametrelerinin otomatik olarak ayarlanması.
7. **Overfitting/Underfitting Analizi**: Modelin aşırı/az öğrenme durumlarının analizi.

Her bölümün başında kısa açıklamalar ve kod blokları ile ilerleyebilirsiniz.

## 1. Veri Önişleme ve Görselleştirme

- Görüntülerin uygun formata dönüştürülmesi (224x224, RGB)
- Etiketleme ve temizleme
- Eğitim/Doğrulama/Test ayrımı
- Sınıf dağılımı ve örnek görsellerin görselleştirilmesi

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Veri dizini ve sınıf isimleri
veri_dizini = "Airbus/aircraft/detection dataset"
siniflar = sorted(os.listdir(veri_dizini))
gorseller, etiketler = [], []

# Görüntüleri yükle ve etiketle
for etiket, sinif_adi in enumerate(siniflar):
    sinif_klasoru = os.path.join(veri_dizini, sinif_adi)
    for dosya_adi in os.listdir(sinif_klasoru):
        yol = os.path.join(sinif_klasoru, dosya_adi)
        img = cv2.imread(yol)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (224, 224))
            gorseller.append(img)
            etiketler.append(etiket)

gorseller = np.array(gorseller)
etiketler = np.array(etiketler)

# Eğitim/Doğrulama/Test ayrımı
X_train, X_temp, y_train, y_temp = train_test_split(gorseller, etiketler, test_size=0.3, stratify=etiketler, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# Sınıf dağılımı görselleştirme
plt.figure(figsize=(8,4))
plt.hist(etiketler, bins=len(siniflar), rwidth=0.8)
plt.title("Sınıf Dağılımı")
plt.xlabel("Sınıf")
plt.ylabel("Adet")
plt.xticks(np.arange(len(siniflar)), siniflar, rotation=45)
plt.show()

In [ ]:
# Eğitim setinden rastgele 5 örnek görsel gösterelim
deneme_indeksler = np.random.choice(len(X_train), 5, replace=False)
fig, axes = plt.subplots(1, 5, figsize=(15,3))
for i, idx in enumerate(deneme_indeksler):
    axes[i].imshow(X_train[idx])
    axes[i].set_title(f"Sınıf: {siniflar[y_train[idx]]}")
    axes[i].axis('off')
plt.suptitle("Örnek Eğitim Görselleri")
plt.show()

## 2. Data Augmentation (Veri Çoğaltma)

Veri artırma ile modelin genelleme başarısı artırılır. Keras ImageDataGenerator ile örnek dönüşümler: Rotation, Flip, Zoom, Brightness.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8,1.2]
)

# Augmentation örneklerini görselleştir
ornek = X_train[0].reshape((1,)+X_train[0].shape)
fig, axes = plt.subplots(1, 5, figsize=(15,3))
i = 0
for batch in datagen.flow(ornek, batch_size=1):
    axes[i].imshow(batch[0].astype('uint8'))
    axes[i].axis('off')
    i += 1
    if i == 5:
        break
plt.suptitle("Augmentation Örnekleri")
plt.show()

## 3. CNN Modeli Oluşturma ve Eğitme

Aşağıda, Keras ile temel bir CNN modeli oluşturulmakta ve eğitim/validasyon setleriyle eğitilmektedir. Modelde Conv2D, MaxPooling2D, Dropout ve Dense katmanları kullanılmıştır.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(siniflar), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    X_train, y_train,
    epochs=15,
    batch_size=32,
    validation_data=(X_val, y_val)
)

## 4. Model Değerlendirme

- Accuracy ve Loss grafikleri (epoch bazında)
- Confusion Matrix ve Classification Report
- Sınıflandırma başarımını detaylı analiz edin

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Accuracy ve Loss grafikleri
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Eğitim')
plt.plot(history.history['val_accuracy'], label='Doğrulama')
plt.title('Doğruluk (Accuracy)')
plt.legend()
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Eğitim')
plt.plot(history.history['val_loss'], label='Doğrulama')
plt.title('Kayıp (Loss)')
plt.legend()
plt.show()

# Confusion Matrix ve Classification Report
y_pred = model.predict(X_test).argmax(axis=1)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', xticklabels=siniflar, yticklabels=siniflar, cmap='Blues')
plt.xlabel('Tahmin')
plt.ylabel('Gerçek')
plt.title('Confusion Matrix')
plt.show()

print(classification_report(y_test, y_pred, target_names=siniflar))

## 5. Grad-CAM ile Görselleştirme

Modelin hangi bölgelere odaklandığını gösteren Grad-CAM örneği. (tf-keras-vis veya benzeri kütüphane ile)

In [ ]:
# Grad-CAM için tf-keras-vis kullanılabilir. Kurulum: pip install tf-keras-vis
from tf_keras_vis.gradcam import Gradcam
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore
import tensorflow as tf

def model_modifier(m):
    m.layers[-1].activation = tf.keras.activations.linear
    return m

gradcam = Gradcam(model, model_modifier=model_modifier)
score = CategoricalScore([y_test[0]])
cam = gradcam(score, X_test[0:1], penultimate_layer=-1)

plt.imshow(X_test[0].astype('uint8'))
plt.imshow(cam[0], cmap='jet', alpha=0.5)
plt.title('Grad-CAM')
plt.axis('off')
plt.show()

## 6. Hiperparametre Optimizasyonu

Keras Tuner ile hiperparametre arama örneği. Katman sayısı, filtre sayısı, dropout oranı, dense boyutu gibi parametreler optimize edilebilir.

In [ ]:
import keras_tuner as kt

def build_model(hp):
    model = Sequential()
    model.add(Conv2D(hp.Int('filters', 32, 128, step=32), (3,3), activation='relu', input_shape=(224,224,3)))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(hp.Float('dropout', 0.2, 0.5, step=0.1)))
    model.add(Flatten())
    model.add(Dense(hp.Int('dense_units', 64, 256, step=64), activation='relu'))
    model.add(Dense(len(siniflar), activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5)
tuner.search(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

best_model = tuner.get_best_models(1)[0]

## 7. Overfitting/Underfitting Analizi

Eğitim ve doğrulama başarı/kayıp grafiklerini inceleyerek modelin aşırı öğrenme (overfitting) veya yetersiz öğrenme (underfitting) durumlarını tespit edebilirsiniz. Eğer eğitim başarısı yüksek, doğrulama başarısı düşükse model overfit olmuştur. Her iki başarı da düşükse underfit olabilir.

Eğitim ve doğrulama başarı/kayıp grafiklerini inceleyerek modelin aşırı öğrenme (overfitting) veya yetersiz öğrenme (underfitting) durumlarını tespit edebilirsiniz. Eğer eğitim başarısı yüksek, doğrulama başarısı düşükse model overfit olmuştur. Her iki başarı da düşükse underfit olabilir.